In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path,'rb') as f:
        ((x_train,y_train),(x_valid,y_valid),_) = pickle.load(f,encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x,m,s): return(x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train,train_mean,train_std)
x_valid = normalize(x_valid,train_mean,train_std)

In [6]:
n,m = x_train.shape
c = y_train.max()+1

## Basic architecture

In [7]:
#num hidden
nh = 50

In [8]:
#simplified kaiming init/ he init
w1 = torch.randn(m,nh)*math.sqrt(2/m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)*math.sqrt(2/m)
b2 = torch.zeros(1)

In [9]:
def lin(x,w,b): return x@w+b
def relu(x): return x.clamp_min(0.)-0.5

In [10]:
t = lin(x_valid,w1,b1)

In [11]:
t.mean(),t.std()

(tensor(-0.0195), tensor(1.4199))

In [12]:
t = relu(t)

In [13]:
#export
from torch.nn import init

In [14]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode="fan_out")
t = relu(lin(x_valid,w1,b1))

In [15]:
def model(x):
    l1 = lin(x,w1,b1)
    l2 = relu(l1)
    return lin(l2,w2,b2)

In [16]:
assert model(x_valid)[:,0].shape==y_valid.shape

## Loss function: MSE

In [17]:
model(x_valid).shape

torch.Size([10000, 1])

In [18]:
#export
def mse(output,targ): return (output.squeeze(-1)-targ).pow(2).mean()

In [19]:
y_train, y_valid = y_train.float(),y_valid.float()

In [20]:
preds = model(x_train)

In [21]:
mse(preds, y_train)

tensor(26.3034)

## Gradients and backward pass

In [22]:
def mse_grad(inp,targ):
    inp.g = 2*(inp-targ).unsqueeze(-1) / inp.shape[0]

In [23]:
def relu_grad(inp, out): 
    inp.g = (inp>0).float() * out.g

In [24]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [25]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

## Refactor model

### Layers as classes

In [26]:
class Relu():
    def __call__(self,inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g
        
class Lin():
    def __init__(self,w,b): self.w,self.b = w,b
        
    def __call__(self,inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [27]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze()-targ).pow(2)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [28]:
class Model():
    def __init__(self,w1,b1,w2,b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
    
    def __call__(self,x,targ):
        for l in self.layers: x = l(x)
        return self.loss(x,targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [29]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1,b1,w2,b2)

In [30]:
%time loss = model(x_train, y_train)

CPU times: user 194 ms, sys: 104 µs, total: 194 ms
Wall time: 48.4 ms


In [31]:
%time model.backward()

CPU times: user 5.2 s, sys: 2.25 s, total: 7.44 s
Wall time: 1.87 s


### Module.forward()

In [32]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [33]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [34]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self,inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @self.w.t()
        self.w.g = torch.einsum("bj,bi -> ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [35]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [36]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [37]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [38]:
%time loss = model(x_train, y_train)

CPU times: user 212 ms, sys: 611 µs, total: 212 ms
Wall time: 52.9 ms


In [39]:
%time model.backward()

CPU times: user 671 ms, sys: 58.7 ms, total: 730 ms
Wall time: 223 ms


### nn.Linear and nn.Module

In [40]:
#export
from torch import nn

In [41]:
class Model (nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(),targ)

In [42]:
model = Model(m, nh, 1)

In [43]:
%time loss = model(x_train, y_train.float())

CPU times: user 116 ms, sys: 723 µs, total: 117 ms
Wall time: 29.5 ms


In [44]:
%time loss.backward()

CPU times: user 338 ms, sys: 117 ms, total: 455 ms
Wall time: 76.9 ms


In [45]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
